<a href="https://colab.research.google.com/github/shwetangmishra1/Test/blob/master/keras_multilabel_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Imports
import tensorflow as tf
from imutils import paths
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
#from pyimagesearch.smallervggnet import SmallerVGGNet
import matplotlib.pyplot as plt
from imutils import paths
import numpy as np
import argparse
import random
import pickle
import cv2
import os
import matplotlib
matplotlib.use("Agg")
# import the necessary packages
#from keras.preprocessing.image import img_to_array
from keras.models import load_model
import imutils

Using TensorFlow backend.


In [0]:
# Check for GPU 
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  #raise SystemError('GPU device not found')
  print("No gpu device")
else:
  print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
######## downloading dataset from drive to colab########
!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
# 1. Authentinon_ receipte and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
# choose a local (colab) directory to store the data.
# local_download_path = os.path.expanduser('~/content')
# print(local_download_path)
# try:
#   os.makedirs(local_download_path)
# except: pass

/root/content


In [0]:
# 2. Auto-iterate using the query syntax
#    https://developers.google.com/drive/v2/web/search-parameters
# file_list = drive.ListFile(
#     {'q': "'1omj-ZSMAR5JIctEQZ4aG-Y4f5i7DYK48' in parents"}).GetList()

# print(file_list)

#file_id = '1omj-ZSMAR5JIctEQZ4aG-Y4f5i7DYK48' # URL id. for fashion dataset
file_id = '1PNvLdRyYGfwUxnSOF7xy7ztc8-gArGPh' # url id for stainless steel datset
file_id_tsv_file = '1PNvLdRyYGfwUxnSOF7xy7ztc8-gArGPh'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('stainless_steel_final_dataset.zip')
#downloaded.GetContentFile('stainless_labels5.tsv.3split.tsv')


In [0]:
# for f in file_list:
#   # 3. Create & download by id.
#   print('title: %s, id: %s' % (f['title'], f['id']))
#   fname = os.path.join(local_download_path, f['title'])
#   print('downloading to {}'.format(fname))
#   f_ = drive.CreateFile({'id': f['id']})
#   f_.GetContentFile(fname)

In [0]:
# Unzip the zipped data
!sudo apt-get install unzip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-21ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.


In [0]:
# unzip the dataset
# path : content/dataset.zip
#!unzip /content/dataset.zip -d /content/dataset
!unzip /content/stainless_steel_final_dataset.zip -d /content

Archive:  /content/stainless_steel_final_dataset.zip
replace /content/stainless_steel_final_dataset/stainless_labels5.tsv.3split.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [0]:
# import the necessary packages
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K

class SmallerVGGNet:
	@staticmethod
	def build(width, height, depth, classes, finalAct="softmax"):
		# initialize the model along with the input shape to be
		# "channels last" and the channels dimension itself
		model = Sequential()
		inputShape = (height, width, depth)
		chanDim = -1

		# if we are using "channels first", update the input shape
		# and channels dimension
		if K.image_data_format() == "channels_first":
			inputShape = (depth, height, width)
			chanDim = 1

		# CONV => RELU => POOL
		model.add(Conv2D(32, (3, 3), padding="same",
			input_shape=inputShape))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(3, 3)))
		model.add(Dropout(0.25))

		# (CONV => RELU) * 2 => POOL
		model.add(Conv2D(64, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Conv2D(64, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		model.add(Dropout(0.25))

		# (CONV => RELU) * 2 => POOL
		model.add(Conv2D(128, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Conv2D(128, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		model.add(Dropout(0.25))

		# first (and only) set of FC => RELU layers
		model.add(Flatten())
		model.add(Dense(1024))
		model.add(Activation("relu"))
		model.add(BatchNormalization())
		model.add(Dropout(0.5))

		# softmax classifier
		model.add(Dense(classes))
		model.add(Activation(finalAct))

		# return the constructed network architecture
		return model

In [0]:
#Set all the necessary paths.
#dataset_path = "/content/dataset/dataset/"
dataset_path = "/content/stainless_steel_final_dataset/stainless_images5"
#output_model_path = "/content/fashion.model"
output_model_path = "/content/ssteel.model"
#labelbin = "/content/mlb.pickle"
plot = "/content/plot_ssteel.png"
labelbin = "/content/steel_mlb.pickle"

In [0]:
# initialize the number of epochs to train for, initial learning rate,
# batch size, and image dimensions
EPOCHS = 75
INIT_LR = 1e-3
BS = 32
IMAGE_DIMS = (256, 256, 3)


In [0]:
data = []
labels = []

In [0]:
# grab the image paths and randomly shuffle them
# print("[INFO] loading images...")
# imagePaths = sorted(list(paths.list_images(dataset_path)))
# print(len(imagePaths))
# assert(len(imagePaths)== 2165)
# random.seed(42)
# random.shuffle(imagePaths)



In [0]:
# loop over the input images
#for imagePath in imagePaths:
	# load the image, pre-process it, and store it in the data list
# 	image = cv2.imread(imagePath)
# 	image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
# 	image = img_to_array(image)
# 	data.append(image)

	# extract set of class labels from the image path and update the
	# labels list
# 	l = label = imagePath.split(os.path.sep)[-2].split("_")
# 	labels.append(l)



In [0]:
import pandas as pd

In [0]:
# getting all the labels
df = pd.read_csv('/content/stainless_steel_final_dataset/stainless_labels5.tsv.3split.tsv',sep = '\t')
df.head()

,ImageName,Labels,dataset_type
0,0006-5519593-20161101-7341-E-Dreyfus-Avenue-12...,"stainless steel fridge,stainless steel oven,st...",train
1,0006-5516609-20161026-10026-W-Potter-Drive-22.jpg,NaN,train
2,0006-5521796-20161106-7516-E-Woodshire-Cove-1.jpg,stainless steel Dishwasher,train
3,0006-5507235-20161006-1725-W-Burnside-Trail-7.jpg,"stainless steel fridge,stainless steel Chimney",train
4,0006-5480519-20160805-2255-E-Squaw-Peak-Drive-...,stainless steel Dishwasher,train


In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10527 entries, 0 to 10526
Data columns (total 3 columns):
ImageName       10527 non-null object
Labels          5759 non-null object
dataset_type    10527 non-null object
dtypes: object(3)
memory usage: 246.8+ KB


In [0]:
# df = df[pd.notnull(df['Labels'])]
# df.info()

In [0]:
df.columns

Index(['ImageName', 'Labels', 'dataset_type'], dtype='object')

In [0]:
imagenames = list(df.iloc[:,0])[:3000]
len(imagenames)

3000

In [0]:
labels = list(df.iloc[:,1])[:3000]
len(labels)

3000

In [0]:
labels[:5]

['stainless steel fridge,stainless steel oven,stainless steel Microwave',
 nan,
 'stainless steel Dishwasher',
 'stainless steel fridge,stainless steel Chimney',
 'stainless steel Dishwasher']

In [0]:
# TO-DO : get all image paths and shuffle them
imagePaths = [os.path.join("/content/stainless_steel_final_dataset/stainless_images5/",imgname) for imgname in imagenames]
print(len(imagePaths))
assert os.path.exists(imagePaths[0])
# print(len(imagePaths))
# #assert(len(imagePaths)== 2165)
# random.seed(42)
# random.shuffle(imagePaths)
# print(len(imagePaths))

3000


In [0]:
# TO-DO : get all labels for images from the data file 
for imagePath in imagePaths:
  image = cv2.imread(imagePath)
  image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
  image = img_to_array(image)
  data.append(image)

In [0]:
print(len(data))

3000


In [0]:
labels[:5]

['stainless steel fridge,stainless steel oven,stainless steel Microwave',
 nan,
 'stainless steel Dishwasher',
 'stainless steel fridge,stainless steel Chimney',
 'stainless steel Dishwasher']

In [0]:
import math

In [0]:
len(labels)

3000

In [0]:
labels1 = []
for val in labels:
  #print(val)
  temp = []
  if str(val) == 'nan':
    labels1.append(temp)
    continue
  values = val.split(',')
  for v in values:
    #print(v)
    tval = v.replace(" ","_")
    temp.append(tval)
  labels1.append(temp)  
print(len(labels1))  
print(labels1[:5])
  

3000
[['stainless_steel_fridge', 'stainless_steel_oven', 'stainless_steel_Microwave'], [], ['stainless_steel_Dishwasher'], ['stainless_steel_fridge', 'stainless_steel_Chimney'], ['stainless_steel_Dishwasher']]


In [0]:
labels1[0]

['stainless_steel_fridge', 'stainless_steel_oven', 'stainless_steel_Microwave']

In [0]:
#df.index
#df.iloc[0]

In [0]:
# for image_path in imagePaths[:2]:
#   image_name = image_path.split('/')[-1]
#   #print(image_name)
#   label = df[image_name][Labels]
#   print(label)

In [0]:
print("No of image files {} and no of labels {}".format(len(data),len(labels1)))
#print(data[0],labels1[0])

No of image files 3000 and no of labels 3000


In [0]:
# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0
labels1 = np.array(labels1)
print("[INFO] data matrix: {} images ({:.2f}MB)".format(len(imagePaths), data.nbytes / (1024 * 1000.0)))

[INFO] data matrix: 3000 images (4608.00MB)


In [0]:
# binarize the labels using scikit-learn's special multi-label
# binarizer implementation
print("[INFO] class labels:")
mlb = MultiLabelBinarizer()
labels1 = mlb.fit_transform(labels1)

[INFO] class labels:


In [0]:
print(mlb.classes_)

['stainless_steel_Chimney' 'stainless_steel_Dishwasher'
 'stainless_steel_Microwave' 'stainless_steel_Sink'
 'stainless_steel_Stove' 'stainless_steel_fridge' 'stainless_steel_oven']


In [0]:
# loop over each of the possible class labels and show them
# for (i, label) in enumerate(mlb.classes_):
# 	print("{}. {}".format(i + 1, labels1))

In [0]:
labels1[1]

array([0, 0, 0, 0, 0, 0, 0])

In [0]:
# partition the data into training and testing splits using 80% of
# the data for training and the remaining 20% for testing
(trainX, testX, trainY, testY) = train_test_split(data,labels1, test_size=0.2, random_state=42)


In [0]:
# construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,horizontal_flip=True, fill_mode="nearest")

In [0]:
# initialize the model using a sigmoid activation as the final layer
# in the network so we can perform multi-label classification
print("[INFO] compiling model...")
model = SmallerVGGNet.build(width=IMAGE_DIMS[1], height=IMAGE_DIMS[0],depth=IMAGE_DIMS[2], classes=len(mlb.classes_),finalAct="sigmoid")

[INFO] compiling model...


In [0]:
# initialize the optimizer (SGD is sufficient)
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)

In [0]:
# compile the model using binary cross-entropy rather than
# categorical cross-entropy -- this may seem counterintuitive for
# multi-label classification, but keep in mind that the goal here
# is to treat each output label as an independent Bernoulli
# distribution
model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])

In [0]:
# train the network
print("[INFO] training network...")
H = model.fit_generator(
	aug.flow(trainX, trainY, batch_size=BS),
	validation_data=(testX, testY),
	steps_per_epoch=len(trainX) // BS,
	epochs=EPOCHS, verbose=1)

[INFO] training network...
Epoch 1/75
263/263 [==============================] - 32s 121ms/step - loss: 0.5924 - acc: 0.7370 - val_loss: 0.6586 - val_acc: 0.7076
Epoch 2/75
263/263 [==============================] - 25s 94ms/step - loss: 0.4880 - acc: 0.8028 - val_loss: 2.1043 - val_acc: 0.6820
Epoch 3/75
263/263 [==============================] - 26s 97ms/step - loss: 0.4670 - acc: 0.8095 - val_loss: 0.4803 - val_acc: 0.7945
Epoch 4/75
263/263 [==============================] - 25s 94ms/step - loss: 0.4516 - acc: 0.8162 - val_loss: 0.4548 - val_acc: 0.8097
Epoch 5/75
263/263 [==============================] - 25s 96ms/step - loss: 0.4361 - acc: 0.8199 - val_loss: 0.5399 - val_acc: 0.8175
Epoch 6/75
263/263 [==============================] - 25s 93ms/step - loss: 0.4251 - acc: 0.8253 - val_loss: 0.7294 - val_acc: 0.7024
Epoch 7/75
263/263 [==============================] - 26s 97ms/step - loss: 0.4200 - acc: 0.8274 - val_loss: 0.4204 - val_acc: 0.8295
Epoch 8/75
263/263 [==============

In [0]:
# save the model to disk
print("[INFO] serializing network...")
model.save(output_model_path)


[INFO] serializing network...


In [0]:
# save the multi-label binarizer to disk
print("[INFO] serializing label binarizer...")
f = open(labelbin, "wb")
f.write(pickle.dumps(mlb))
f.close()

[INFO] serializing label binarizer...


In [0]:
# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
N = EPOCHS
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="upper left")
plt.savefig(plot)

In [0]:
#img_path = "/content/example_05.jpg"
img_path = "/content/testimg.jpg"
image = cv2.imread(img_path)
output = imutils.resize(image, width=400)
 

In [0]:
#import matplotlib.pyplot as plt

In [0]:
#plt.imshow(np.array(output))

In [0]:
# pre-process the image for classification
image = cv2.resize(image, (96, 96))
image = image.astype("float") / 255.0
image = img_to_array(image)
image = np.expand_dims(image, axis=0)

In [0]:
print("[INFO] loading network...")
model_ = "/content/ssteel.model"
model = load_model(model_)
labelbin_ = "/content/steel_mlb.pickle"
mlb = pickle.loads(open(labelbin_, "rb").read())

[INFO] loading network...


In [0]:
# classify the input image then find the indexes of the two class
# labels with the *largest* probability
print("[INFO] classifying image...")
proba = model.predict(image)[0]
idxs = np.argsort(proba)[::-1][:2]

[INFO] classifying image...


In [0]:
# loop over the indexes of the high confidence class labels
for (i, j) in enumerate(idxs):
	# build the label and draw the label on the image
	label = "{}: {:.2f}%".format(mlb.classes_[j], proba[j] * 100)
	cv2.putText(output, label, (10, (i * 30) + 25), 
		cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

stainless_steel_Chimney: 78.78%
stainless_steel_Dishwasher: 1.25%
stainless_steel_Microwave: 5.17%
stainless_steel_Sink: 30.96%
stainless_steel_Stove: 47.70%
stainless_steel_fridge: 90.79%
stainless_steel_oven: 22.22%

In [0]:
# show the probabilities for each of the individual labels
for (label, p) in zip(mlb.classes_, proba):
	print("{}: {:.2f}%".format(label, p * 100))


stainless_steel_Chimney: 7.61%
stainless_steel_Dishwasher: 23.71%
stainless_steel_Microwave: 30.70%
stainless_steel_Sink: 19.92%
stainless_steel_Stove: 9.61%
stainless_steel_fridge: 18.70%
stainless_steel_oven: 36.97%


In [0]:
# show the output image
# cv2.imshow("Output", output)
# cv2.waitKey(0)

In [0]:
# Lets train Inception v3

In [0]:
from keras import backend as K
from keras.applications.inception_v3 import InceptionV3
from keras.layers import *
from keras.models import Model
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator

import numpy as np

In [0]:
base_model = InceptionV3(weights='imagenet', include_top=False)

Instructions for updating:
Colocations handled automatically by placer.
87916544/87910968 [==============================] - 3s 0us/step


In [0]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
x = Dropout(0.3)(x)
predictions = Dense(7, activation='sigmoid')(x)

 # this is the model we will train
model = Model(input=base_model.input, output=predictions)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  # Remove the CWD from sys.path while we load stuff.


In [0]:
for layer in base_model.layers:
  layer.trainable = False


In [0]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=["accuracy"])

In [0]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, None, 3 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [0]:
len(trainX), len(testX),len(trainY), len(testY)

(2400, 600, 2400, 600)

In [0]:
trainX.shape, testX.shape, trainY.shape, testY.shape

((2400, 256, 256, 3), (600, 256, 256, 3), (2400, 7), (600, 7))

In [0]:
trainY[0]

array([1, 0, 1, 0, 0, 0, 0])

In [0]:
print("[INFO] training network...")
H = model.fit_generator(
	aug.flow(trainX, trainY, batch_size= BS),
	validation_data=(testX, testY),
	steps_per_epoch=len(trainX) // BS,
	epochs=250, verbose=1)

[INFO] training network...


NameError: ignored

In [0]:
# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 172 layers and unfreeze the rest:
for layer in model.layers[:172]:
  layer.trainable = False
for layer in model.layers[172:]:
  layer.trainable = True

In [0]:
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9),loss='binary_crossentropy',metrics=["accuracy"])

In [0]:
H = model.fit_generator(
	aug.flow(trainX, trainY, batch_size=BS),
	validation_data=(testX, testY),
	steps_per_epoch=len(trainX) // BS,
	epochs=EPOCHS, verbose=1)

Epoch 1/75
263/263 [==============================] - 45s 173ms/step - loss: 0.4401 - acc: 0.8213 - val_loss: 0.4508 - val_acc: 0.8169
Epoch 2/75
263/263 [==============================] - 38s 143ms/step - loss: 0.4395 - acc: 0.8213 - val_loss: 0.4505 - val_acc: 0.8169
Epoch 3/75
263/263 [==============================] - 37s 141ms/step - loss: 0.4396 - acc: 0.8216 - val_loss: 0.4501 - val_acc: 0.8169
Epoch 4/75
263/263 [==============================] - 37s 142ms/step - loss: 0.4388 - acc: 0.8216 - val_loss: 0.4505 - val_acc: 0.8169
Epoch 5/75
263/263 [==============================] - 37s 141ms/step - loss: 0.4398 - acc: 0.8214 - val_loss: 0.4501 - val_acc: 0.8170
Epoch 6/75
263/263 [==============================] - 37s 140ms/step - loss: 0.4377 - acc: 0.8217 - val_loss: 0.4501 - val_acc: 0.8170
Epoch 7/75
263/263 [==============================] - 37s 141ms/step - loss: 0.4403 - acc: 0.8200 - val_loss: 0.4509 - val_acc: 0.8170
Epoch 8/75
263/263 [==============================] - 3